In [26]:
from transformers import AutoTokenizer, AutoModel
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch
from torch import nn

import pandas as pd

In [27]:
model = AutoModel.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

/home/penta-101858/Code/VQA/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
original_sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু"
fake_sentence = "আপনি হতাশ কারণ  আমার জন্য অনেক কিছু করেছেন।"
fake_sentence = normalize(fake_sentence) # this normalization step is required before tokenizing the text
original_sentence = normalize(original_sentence)

In [29]:
fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode_plus(fake_sentence, return_tensors="pt")
# fake_inputs

original_inputs = tokenizer.encode_plus(original_sentence, return_tensors="pt")

In [30]:
discriminator_outputs = model(fake_inputs['input_ids'],attention_mask= fake_inputs['attention_mask'])#.logits
discriminator_outputs2 = model(original_inputs['input_ids'],attention_mask= original_inputs['attention_mask'])#.logits
# predictions = torch.round((torch.sign(discriminator_outputs) + 1) / 2)

In [31]:
input1 = discriminator_outputs['last_hidden_state'][:,0,:]
input2 = discriminator_outputs2['last_hidden_state'][:,0,:]

In [32]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [33]:
output = cos(input1, input2)
output

tensor([0.8526], grad_fn=<SumBackward1>)

In [34]:
df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)

/tmp/ipykernel_84842/2331687903.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
/tmp/ipykernel_84842/2331687903.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)


In [35]:
def consolidate(gdf):
    captions = gdf['caption'].to_list()
    return captions

df1 = df_train.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})
df2 = df_test.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})

In [36]:
df1.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)
df2.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)

df = pd.concat([df1, df2], axis=0)
df = df.reset_index(drop=True)

In [37]:
df.head()

,image_name,captions
0,1.jpg,"[একটি পার্কে একটি লেক আছে ।, লেকের উপর একটি ব্..."
1,2.jpg,"[একজন মানুষ মাথায় খড় নিয়ে হেটে যাচ্ছে ।, এক..."
2,3.jpg,"[টেবিলের উপর একটি চশমা আছে ।, টেবিলের উপর একটি..."
3,4.jpg,"[একটি শিশু দেখা যাচ্ছে ।, একটি শিশু গ্রামের ছো..."
4,5.jpg,"[একটি চায়ের কাপ দেখা যাচ্ছে ।, চায়ের কাপে উপ..."


In [38]:
import itertools

In [48]:
def top_3_captions(captions: list):
    """
    top 3 being the max length string, minimum length string and the string which has maximum cosine similarity index with 
    both of the max and min length string

    Args:
        captions (list): list of 5 captions from the bornon dataset for each image

    Returns:
        _type_: list of top 3 captions
    """
    cap_discriminator_outputs = []
    cap_inputs = []
    cap_final = []
    
    with torch.no_grad():
    # cap_final.append(tokenizer.encode_plus(normalize(max(captions, key=len)), return_tensors="pt"))
    # cap_final.append(tokenizer.encode_plus(normalize(min(captions, key=len)), return_tensors="pt"))
        cap_final.append(normalize(max(captions, key=len)))  
        cap_final.append(normalize(min(captions, key=len)))  
        
        for i in range(len(captions)):
            normalised = normalize(captions[i])
            cap_inputs.append(tokenizer.encode_plus(normalised, return_tensors="pt"))
        
        for i in range(len(cap_inputs)):
            cap_discriminator_outputs.append(model(cap_inputs[i]['input_ids'],attention_mask= cap_inputs[i]['attention_mask'])['last_hidden_state'][:,0,:])
        
        max_len_str = tokenizer.encode_plus(cap_final[0], return_tensors="pt")
        min_len_str = tokenizer.encode_plus(cap_final[1], return_tensors="pt")
        
        
        max_len_str = model(max_len_str['input_ids'],attention_mask= max_len_str['attention_mask'])['last_hidden_state'][:,0,:]
        min_len_str = model(min_len_str['input_ids'],attention_mask= min_len_str['attention_mask'])['last_hidden_state'][:,0,:]
        
        
        # print(max_len_str.shape)
        
        max_val = -1
        idx = 0
        for i in range(len(cap_discriminator_outputs)):
            # print(i)
            if cos(max_len_str, cap_discriminator_outputs[i]) > 0.99 or cos(min_len_str, cap_discriminator_outputs[i]) > 0.99:
                # print(captions[i], cap_final[0], cap_final[1])
                print("max: ", cos(max_len_str, cap_discriminator_outputs[i]), "min: ", cos(min_len_str, cap_discriminator_outputs[i]))
                
            else:
                val1 = cos(max_len_str, cap_discriminator_outputs[i])
                val2 = cos(min_len_str, cap_discriminator_outputs[i])
                
                # print(captions[i])
                # print(captions[i], cap_final[0], cap_final[1])
                
                if val1+val2 > max_val:
                    max_val = val1+val2
                    idx = i
                    
        cap_final.append(normalize(captions[idx]))
        
    return (cap_final)
    

In [49]:
df['top_captions'] = None
from tqdm import notebook as nb

In [50]:
for idx, row in nb.tqdm(df.iterrows()):
    
    outs = top_3_captions(captions=row['captions'])
    # df.loc[idx, 'top_captions'] = ',\n '.join(map(str, outs))
    
    df.loc[df['image_name'] == row['image_name'], 'top_captions'] = ',\n '.join(map(str, outs))
    

0it [00:00, ?it/s]

max:  tensor([0.8280]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.8280])
max:  tensor([0.9723]) min:  tensor([1.0000])
max:  tensor([1.]) min:  tensor([0.9723])
max:  tensor([0.9137]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.9137])
max:  tensor([0.8906]) min:  tensor([1.0000])
max:  tensor([1.]) min:  tensor([0.8906])
max:  tensor([0.9477]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.9477])
max:  tensor([0.9032]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.9032])
max:  tensor([0.8685]) min:  tensor([1.])
max:  tensor([1.0000]) min:  tensor([0.8685])
max:  tensor([0.8604]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.8604])
max:  tensor([0.8818]) min:  tensor([1.])
max:  tensor([1.0000]) min:  tensor([0.8818])
max:  tensor([0.9087]) min:  tensor([1.0000])
max:  tensor([1.]) min:  tensor([0.9087])
max:  tensor([0.9926]) min:  tensor([0.9194])
max:  tensor([0.9367]) min:  tensor([1.])
max:  tensor

In [53]:
df.head(10)

,image_name,captions,top_captions
0,1.jpg,"[একটি পার্কে একটি লেক আছে ।, লেকের উপর একটি ব্...","গাছপালায় ঘেরা এই পার্কটি খুবই মনমুগ্ধকর ।,\n ..."
1,2.jpg,"[একজন মানুষ মাথায় খড় নিয়ে হেটে যাচ্ছে ।, এক...",লুঙ্গি পড়া একজন মানুষ খড়ের উপর হেটে যাচ্ছে ।...
2,3.jpg,"[টেবিলের উপর একটি চশমা আছে ।, টেবিলের উপর একটি...","একটি রুমে একটি খাট এবং একটি টেবিল আছে,টেবিলের ..."
3,4.jpg,"[একটি শিশু দেখা যাচ্ছে ।, একটি শিশু গ্রামের ছো...",একটি শিশু হাটার সময় তার কিছুটা ছায়া দেখা যাচ...
4,5.jpg,"[একটি চায়ের কাপ দেখা যাচ্ছে ।, চায়ের কাপে উপ...","আলোর কারনে চায়ের কাপের ছায়া দেখা যাচ্ছে ।,\n..."
5,6.jpg,"[তিনটি শিশু দেখা যাচ্ছে ।, একটি ধানের ক্ষেত দে...",ধানের ক্ষেতের পিছনে অনেক গাছপালা দেখা যাচ্ছে ।...
6,7.jpg,"[একটি লোক নৌকার উপর দাড়িয়ে আছে ।, একটি লোক ক...",একটি লোক দেখা যাচ্ছে এবং লেকের আশেপাশে অনেকগুল...
7,8.jpg,"[একটি পার্ক দেখা যাচ্ছে ।, একটি পার্কে একজন শি...",একটি শিশু ছাতা মাথায় নিয়ে এবং একজন লোক মাথায...
8,9.jpg,"[৮ জন শিশু দেখা যাচ্ছে ।, ৮ জন শিশুকে হাসতে দে...",৭ জন শিশু খালি গায়ে দেখা যাচ্ছে এবং একজনের গা...
9,10.jpg,"[একটি লোহার জালি দেখা যাচ্ছে ।, একটি জালির পেছ...",একটি জালির পেছনে একটি রিক্সা এবং একটি গাড়ি দে...


In [54]:
df.to_csv("top_captioned.csv", index=False)